In [1]:
%run 2021_11_02_Psilocybin_in_Cg_Exchange_Function.ipynb #change model source here

No objective coefficients in model. Unclear what should be optimized


In [2]:
from cobra.io import read_sbml_model
from cobra.medium import minimal_medium
import pandas as pd
import matplotlib.pyplot as plt

#max_growth = model.slim_optimize()
#model.medium = minimal_medium(model, max_growth)

In [3]:
target = model.reactions.EX_psi_e

In [4]:
#model.reactions.EX_psi_e

In [5]:
#model.reactions.EX_psi_e.upper_bound=0

In [6]:
with model:
    model.objective = target
    print(model.optimize().objective_value)

5.69299883708285e-05


# Overview of Reactions

In [7]:
#for x in model.reactions:
 #   print("%s : %s" % (x.id, x.reaction))

In [8]:
model.reactions.query('psi_c','reaction')

[<Reaction psiK2 at 0x23f35120588>,
 <Reaction psiM2 at 0x23f351202b0>,
 <Reaction pho at 0x23f35120208>,
 <Reaction psitpp at 0x23f35120550>]

In [9]:
for x in model.reactions.query('trp_L_c','reaction'):
    print("%s : %s" % (x.id, x.reaction)) #lists all reactions containing tryptophan_c

TRPabcpp : atp_c + h2o_c + trp_L_e --> adp_c + h_c + pi_c + trp_L_c
TRPt2rpp : h_e + trp_L_e <-- h_c + trp_L_c
TRPTRS : atp_c + trnatrp_c + trp_L_c --> amp_c + ppi_c + trptrna_c
TRPS1 : 3ig3p_c + ser_L_c --> g3p_c + h2o_c + trp_L_c
TRPS2 : indole_c + ser_L_c --> h2o_c + trp_L_c
PROTEIN : 1.3527 ala_L_c + 0.38519 arg_L_c + 0.39281 asn_L_c + 0.39281 asp_L_c + 40.0656 atp_c + 0.089437 cys_L_c + 0.69314 gln_L_c + 1.1139 glu_L_c + 0.71651 gly_c + 30.7601 h2o_c + 0.13619 his_L_c + 0.38316 ile_L_c + 0.71347 leu_L_c + 0.37909 lys_L_c + 0.15347 met_L_c + 0.26018 phe_L_c + 0.32319 pro_L_c + 0.498 ser_L_c + 0.5539 thr_L_c + 0.055899 trp_L_c + 0.15753 tyr_L_c + 0.55492 vaL_L_c --> 40.0656 adp_c + 40.0656 h_c + 40.0656 pi_c + protein_c
CrTdc : trp_L_c --> co2_c + tryp_c


In [10]:
print(model.reactions.CrTdc.id, ':', model.reactions.CrTdc.reaction)
print(model.reactions.psiH.id, ':', model.reactions.psiH.reaction)
print(model.reactions.psiK.id, ':', model.reactions.psiK.reaction)
print(model.reactions.psiM.id, ':', model.reactions.psiM.reaction)
print(model.reactions.psiM2.id, ':', model.reactions.psiM2.reaction)

#print(model.reactions.pho.reaction)
#print(model.reactions.psiK2.reaction)

CrTdc : trp_L_c --> co2_c + tryp_c
psiH : 2 nadph_c + o2_c + tryp_c --> _4htryp_c + h2o_c + 2 nadp_c
psiK : _4htryp_c + atp_c --> adp_c + nbc_c
psiM : amet_c + nbc_c --> ahcys_c + baecyst_c
psiM2 : amet_c + baecyst_c --> ahcys_c + psi_c


# Set Bounds Here

In [11]:
#model.reactions.TRPS1.upper_bound=100 #ensure tryptophan production, explore media later. 
#model.reactions.TRPS1.lower_bound=.1

#model.reactions.TRPt2rpp.upper_bound=0 #Knock out tryptophan exporter.
#model.reactions.TRPt2rpp.lower_bound=0
#model.reactions.TRPt2rpp.bounds =0,0

In [12]:
#model.reactions.TRPS1

# Resulting Flux Optimized for Psi Production

There's a lot of me messing around here

In [13]:
model.medium

{'EX_ca2_e': 0.0144877134286247,
 'EX_cobalt2_e': 7.00338234344926e-05,
 'EX_cu2_e': 0.00197271722237563,
 'EX_fe2_e': 0.16239125,
 'EX_fe3_e': 0.0217300637039616,
 'EX_glc_e': 4.67,
 'EX_k_e': 0.543285250115532,
 'EX_mg2_e': 0.0241459221505149,
 'EX_mn2_e': 0.00192443552016793,
 'EX_nh4_e': 11.9121416666662,
 'EX_ni2_e': 0.00089824250292982,
 'EX_o2_e': 16.5067783333334,
 'EX_pi_e': 0.918000000000888,
 'EX_so4_e': 4.01673156249995,
 'EX_zn2_e': 0.000948926279874219,
 'EX_BIOTIN': 1.01925397503027e-06,
 'EX_cl': 0.0144877134286247,
 'EX_psi_e': 1000.0}

## before messing with media

In [14]:
with model:
    model.objective = target
    print("every reaction going toward psilocybin")
    
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    
    print("Is there glucose uptake?",model.reactions.EX_glc_e.flux)
    print("Is there growth?",model.reactions.CG_biomass_cgl_ATCC13032.flux)
    
    print("tryptamine [mmol gDW^-1 h^-1]:",model.reactions.CrTdc.flux)
    print("4-Hydroxytryptamine [mmol gDW^-1 h^-1]:",model.reactions.psiH.flux)
    print("norbaeocystin [mmol gDW^-1 h^-1]:",model.reactions.psiK.flux)
    print("baeocystin [mmol gDW^-1 h^-1]:",model.reactions.psiM.flux)
    print("psilocybin [mmol gDW^-1 h^-1]:",model.reactions.psiM2.flux)
    print("transport [mmol gDW^-1 h^-1]:",model.reactions.psitpp.flux)
    print("out [mmol gDW^-1 h^-1]:",target.flux)

every reaction going toward psilocybin
Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
Is there glucose uptake? -4.116294619992247
Is there growth? 0.0
tryptamine [mmol gDW^-1 h^-1]: 5.69299883708285e-05
4-Hydroxytryptamine [mmol gDW^-1 h^-1]: 5.69299883708285e-05
norbaeocystin [mmol gDW^-1 h^-1]: 5.69299883708285e-05
baeocystin [mmol gDW^-1 h^-1]: 5.69299883708285e-05
psilocybin [mmol gDW^-1 h^-1]: 5.69299883708285e-05
transport [mmol gDW^-1 h^-1]: 5.69299883708285e-05
out [mmol gDW^-1 h^-1]: 5.69299883708285e-05


In [15]:
with model:
    model.objective = target
    print("every reaction going from Typtophan")
    
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    print("TRPTRS [mmol gDW^-1 h^-1]:",model.reactions.TRPTRS.flux)
    print("TRPS2 [mmol gDW^-1 h^-1]:",model.reactions.TRPS2.flux)
    print("CrTdc [mmol gDW^-1 h^-1]:",model.reactions.CrTdc.flux)
    print("TRPS1 [mmol gDW^-1 h^-1]:",model.reactions.TRPS1.flux)
    print("TRPt2rpp [mmol gDW^-1 h^-1]:",model.reactions.TRPt2rpp.flux)
    print("TRPabcpp [mmol gDW^-1 h^-1]:",model.reactions.TRPabcpp.flux)
    print("PROTEIN [mmol gDW^-1 h^-1]:",model.reactions.PROTEIN.flux)
    
    

every reaction going from Typtophan
Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
TRPTRS [mmol gDW^-1 h^-1]: 0.0
TRPS2 [mmol gDW^-1 h^-1]: 0.0
CrTdc [mmol gDW^-1 h^-1]: 5.69299883708285e-05
TRPS1 [mmol gDW^-1 h^-1]: 5.69299883708285e-05
TRPt2rpp [mmol gDW^-1 h^-1]: 0.0
TRPabcpp [mmol gDW^-1 h^-1]: 0.0
PROTEIN [mmol gDW^-1 h^-1]: 0.0


Note growth rate of zero, as the production of psilocybin will be highest when there is no growth.

### And now if adding demand functions

In [16]:
with model:
    model.objective = target
    print("Demand functions. For some reason, the solver can't actually check any of them,") 
    print("yet if they're not there the fluxes for psilocibin are zero. Strange")

    #print("tryptamine [mmol gDW^-1 h^-1]:",model.reactions.DM_tryp_c.flux)
    #print("4-Hydroxytryptamine [mmol gDW^-1 h^-1]:",model.reactions.DM__4htryp_c.flux)
    #print("norbaeocystin [mmol gDW^-1 h^-1]:",model.reactions.DM_nbc_c.flux)
    #print("baeocystin [mmol gDW^-1 h^-1]:",model.reactions.DM_psi_e.flux)
    #print("psilocybin [mmol gDW^-1 h^-1]:",model.reactions.DM_amet_c.flux)
    #print("transport [mmol gDW^-1 h^-1]:",model.reactions.DM_ahcys_c.flux)
    #print("out [mmol gDW^-1 h^-1]:",model.reactions.DM_psii_c.flux)

Demand functions. For some reason, the solver can't actually check any of them,
yet if they're not there the fluxes for psilocibin are zero. Strange


In [17]:
#model.reactions.query('EX_','id')

In [18]:
#model.reactions.EX_ala_L_e 

In [19]:
#model.reactions.TRPabcpp.lower_bound=10

## Modify Medium

In [20]:
medium = model.medium
medium["EX_glc_e"] = 1000 
medium["EX_o2_e"] = 10
medium["EX_sucr_e"] = 1000
medium["EX_trp_L_e"] = 0
medium["EX_ala_L_e"] = 1000
#medium["EX_psi_e"] = 0


model.medium = medium

model.medium

{'EX_ala_L_e': 1000,
 'EX_ca2_e': 0.0144877134286247,
 'EX_cobalt2_e': 7.00338234344926e-05,
 'EX_cu2_e': 0.00197271722237563,
 'EX_fe2_e': 0.16239125,
 'EX_fe3_e': 0.0217300637039616,
 'EX_glc_e': 1000,
 'EX_k_e': 0.543285250115532,
 'EX_mg2_e': 0.0241459221505149,
 'EX_mn2_e': 0.00192443552016793,
 'EX_nh4_e': 11.9121416666662,
 'EX_ni2_e': 0.00089824250292982,
 'EX_o2_e': 10,
 'EX_pi_e': 0.918000000000888,
 'EX_so4_e': 4.01673156249995,
 'EX_sucr_e': 1000,
 'EX_zn2_e': 0.000948926279874219,
 'EX_BIOTIN': 1.01925397503027e-06,
 'EX_cl': 0.0144877134286247,
 'EX_psi_e': 1000.0}

## Set Bounds Here

In [21]:
model.reactions.TRPS1.upper_bound=100 #ensure tryptophan production, explore media later. 
model.reactions.TRPS1.lower_bound=.1

#model.reactions.TRPt2rpp.upper_bound=0 #Knock out tryptophan exporter.
#model.reactions.TRPt2rpp.lower_bound=0
model.reactions.TRPt2rpp.bounds =0,0.5 #downregulate tryptophan exporter

In [22]:
model.reactions.TRPt2rpp

Reaction identifier,TRPt2rpp
Name,L_tryptophan reversible transport via proton symport
Memory address,0x023f352f7ef0
Stoichiometry,h_e + trp_L_e --> h_c + trp_L_c H+ + L_Tryptophan --> H+ + L_Tryptophan
GPR,cg1257
Lower bound,0
Upper bound,0.5


In [23]:
model.reactions.CG_biomass_cgl_ATCC13032.lower_bound=0.1

## Result From Modified Medium

In [24]:
model.reactions.psiM2

Reaction identifier,psiM2
Name,
Memory address,0x023f351202b0
Stoichiometry,amet_c + baecyst_c --> ahcys_c + psi_c S-AdenosyL_L_methionine + Baeocystin --> S-AdenosyL_L_homocysteine + Psilocybin
GPR,
Lower bound,0.0
Upper bound,1000.0


In [25]:
with model:
    model.objective = target
    print("every reaction going toward psilocybin")
    
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    
    print("Is there glucose uptake?",model.reactions.EX_glc_e.flux)
    print("Is there growth?",model.reactions.CG_biomass_cgl_ATCC13032.flux)
    
    print("tryptamine [mmol gDW^-1 h^-1]:",model.reactions.CrTdc.flux)
    print("4-Hydroxytryptamine [mmol gDW^-1 h^-1]:",model.reactions.psiH.flux)
    print("norbaeocystin [mmol gDW^-1 h^-1]:",model.reactions.psiK.flux)
    print("baeocystin [mmol gDW^-1 h^-1]:",model.reactions.psiM.flux)
    print("psilocybin [mmol gDW^-1 h^-1]:",model.reactions.psiM2.flux)
    print("pho [mmol gDW^-1 h^-1]:",model.reactions.pho.flux)
    print("restore [mmol gDW^-1 h^-1]:",model.reactions.psiK2.flux)
    print("transport [mmol gDW^-1 h^-1]:",model.reactions.psitpp.flux)
    print("out [mmol gDW^-1 h^-1]:",target.flux)

every reaction going toward psilocybin
Max. psi production [mmol gDW^-1 h^-1]: 4.365741867082851e-05
Is there glucose uptake? -4.23869552879857
Is there growth? 0.1
tryptamine [mmol gDW^-1 h^-1]: 0.096979888828
4-Hydroxytryptamine [mmol gDW^-1 h^-1]: 0.096979888828
norbaeocystin [mmol gDW^-1 h^-1]: 4.365741867082851e-05
baeocystin [mmol gDW^-1 h^-1]: 4.365741867082851e-05
psilocybin [mmol gDW^-1 h^-1]: 4.365741867082851e-05
pho [mmol gDW^-1 h^-1]: 0.0
restore [mmol gDW^-1 h^-1]: 0.0
transport [mmol gDW^-1 h^-1]: 4.365741867082851e-05
out [mmol gDW^-1 h^-1]: 4.365741867082851e-05


In [26]:
with model:
    model.objective = target
    print("every reaction going from Typtophan")
    
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    print("TRPTRS [mmol gDW^-1 h^-1]:",model.reactions.TRPTRS.flux)
    print("TRPS2 [mmol gDW^-1 h^-1]:",model.reactions.TRPS2.flux)
    print("CrTdc [mmol gDW^-1 h^-1]:",model.reactions.CrTdc.flux)
    print("TRPS1 [mmol gDW^-1 h^-1]:",model.reactions.TRPS1.flux)
    print("TRPt2rpp [mmol gDW^-1 h^-1]:",model.reactions.TRPt2rpp.flux)
    print("TRPabcpp [mmol gDW^-1 h^-1]:",model.reactions.TRPabcpp.flux)
    print("PROTEIN [mmol gDW^-1 h^-1]:",model.reactions.PROTEIN.flux)
    
    

every reaction going from Typtophan
Max. psi production [mmol gDW^-1 h^-1]: 4.365741867082851e-05
TRPTRS [mmol gDW^-1 h^-1]: 0.0
TRPS2 [mmol gDW^-1 h^-1]: 0.0
CrTdc [mmol gDW^-1 h^-1]: 0.096979888828
TRPS1 [mmol gDW^-1 h^-1]: 0.1
TRPt2rpp [mmol gDW^-1 h^-1]: 0.0
TRPabcpp [mmol gDW^-1 h^-1]: 0.0
PROTEIN [mmol gDW^-1 h^-1]: 0.054028


In [27]:
model.reactions.TRPS1  

Reaction identifier,TRPS1
Name,tryptophan synthase (indoleglycerol phosphate)
Memory address,0x023f35426c18
Stoichiometry,3ig3p_c + ser_L_c --> g3p_c + h2o_c + trp_L_c C'-(3-Indolyl)-glycerol 3-phosphate + L_Serine --> Glyceraldehyde 3-phosphate + H2O + L_Tryptophan
GPR,cg3363 and cg3364
Lower bound,0.1
Upper bound,100


In [28]:
model.reactions.TRPt2rpp #transport of tryptophan from cell

Reaction identifier,TRPt2rpp
Name,L_tryptophan reversible transport via proton symport
Memory address,0x023f352f7ef0
Stoichiometry,h_e + trp_L_e --> h_c + trp_L_c H+ + L_Tryptophan --> H+ + L_Tryptophan
GPR,cg1257
Lower bound,0
Upper bound,0.5


not viable to knock out protein production

In [29]:
model.slim_optimize()

0.0

# Adding amino acids

In [30]:
model.metabolites.query('Alanine','name')

[<Metabolite ala_B_c at 0x23f34ad8438>,
 <Metabolite ala_B_e at 0x23f34ad84e0>,
 <Metabolite ala_D_c at 0x23f34ad85f8>,
 <Metabolite ala_D_e at 0x23f34ad86a0>,
 <Metabolite ala_L_c at 0x23f34ad8748>,
 <Metabolite ala_L_e at 0x23f34ad87f0>]

In [31]:
model.reactions.EX_ala_L_e #AA format

Reaction identifier,EX_ala_L_e
Name,L_Alanine exchange
Memory address,0x023f34f98f98
Stoichiometry,ala_L_e <=> L_Alanine <=>
GPR,
Lower bound,-1000
Upper bound,4.61339666666667


In [39]:
medium = model.medium
medium["EX_ala_L_e"] = 10
medium["EX_arg_L_e"] = 10
medium["EX_asn_L_e"] = 10
medium["EX_asp_L_e"] = 10
medium["EX_cys_L_e"] = 10
medium["EX_gln_L_e"] = 10
medium["EX_glu_L_e"] = 10
medium["EX_gly_e"] = 10
medium["EX_his_L_e"] = 10
medium["EX_ile_L_e"] = 10
medium["EX_leu_L_e"] = 10
medium["EX_lys_L_e"] = 10
medium["EX_met_L_e"] = 10
medium["EX_phe_L_e"] = 10
medium["EX_pro_L_e"] = 10
medium["EX_ser_L_e"] = 10
medium["EX_thr_L_e"] = 10
medium["EX_trp_L_e"] = 10
medium["EX_tyr_L_e"] = 10
medium["EX_val_L_e"] = 10
#these didn't actually seem to help at all

medium["EX_pi_e"] = 10
medium["EX_h_e"] = 10
medium["EX_h_e"] = 10
medium["EX_h_e"] = 10

medium["EX_o2_e"] = 7
medium["EX_nh4_e"] = 7

model.medium = medium

model.medium

{'EX_ala_L_e': 10,
 'EX_arg_L_e': 10,
 'EX_asn_L_e': 10,
 'EX_asp_L_e': 10,
 'EX_ca2_e': 0.0144877134286247,
 'EX_cobalt2_e': 7.00338234344926e-05,
 'EX_cu2_e': 0.00197271722237563,
 'EX_cys_L_e': 10,
 'EX_fe2_e': 0.16239125,
 'EX_fe3_e': 0.0217300637039616,
 'EX_glc_e': 1000,
 'EX_gln_L_e': 10,
 'EX_glu_L_e': 10,
 'EX_gly_e': 10,
 'EX_h_e': 10,
 'EX_his_L_e': 10,
 'EX_ile_L_e': 10,
 'EX_k_e': 0.543285250115532,
 'EX_leu_L_e': 10,
 'EX_lys_L_e': 10,
 'EX_met_L_e': 10,
 'EX_mg2_e': 0.0241459221505149,
 'EX_mn2_e': 0.00192443552016793,
 'EX_nh4_e': 7,
 'EX_ni2_e': 0.00089824250292982,
 'EX_o2_e': 7,
 'EX_phe_L_e': 10,
 'EX_pi_e': 10,
 'EX_pro_L_e': 10,
 'EX_ser_L_e': 10,
 'EX_so4_e': 4.01673156249995,
 'EX_sucr_e': 1000,
 'EX_thr_L_e': 10,
 'EX_trp_L_e': 10,
 'EX_tyr_L_e': 10,
 'EX_val_L_e': 10,
 'EX_zn2_e': 0.000948926279874219,
 'EX_BIOTIN': 1.01925397503027e-06,
 'EX_cl': 0.0144877134286247,
 'EX_psi_e': 1000.0}

In [40]:
with model:
    model.objective = target
    print("every reaction going toward psilocybin")
    
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    
    print("Is there glucose uptake?",model.reactions.EX_glc_e.flux)
    print("Is there growth?",model.reactions.CG_biomass_cgl_ATCC13032.flux)
    
    print("tryptamine [mmol gDW^-1 h^-1]:",model.reactions.CrTdc.flux)
    print("4-Hydroxytryptamine [mmol gDW^-1 h^-1]:",model.reactions.psiH.flux)
    print("norbaeocystin [mmol gDW^-1 h^-1]:",model.reactions.psiK.flux)
    print("baeocystin [mmol gDW^-1 h^-1]:",model.reactions.psiM.flux)
    print("psilocybin [mmol gDW^-1 h^-1]:",model.reactions.psiM2.flux)
    print("pho [mmol gDW^-1 h^-1]:",model.reactions.pho.flux)
    print("restore [mmol gDW^-1 h^-1]:",model.reactions.psiK2.flux)
    print("transport [mmol gDW^-1 h^-1]:",model.reactions.psitpp.flux)
    print("out [mmol gDW^-1 h^-1]:",target.flux)

every reaction going toward psilocybin
Max. psi production [mmol gDW^-1 h^-1]: 4.365741867082851e-05
Is there glucose uptake? -4.220371165592363
Is there growth? 0.1
tryptamine [mmol gDW^-1 h^-1]: 0.096979888828
4-Hydroxytryptamine [mmol gDW^-1 h^-1]: 4.365741867082851e-05
norbaeocystin [mmol gDW^-1 h^-1]: 4.365741867082851e-05
baeocystin [mmol gDW^-1 h^-1]: 4.365741867082851e-05
psilocybin [mmol gDW^-1 h^-1]: 4.365741867082851e-05
pho [mmol gDW^-1 h^-1]: 0.0
restore [mmol gDW^-1 h^-1]: 0.0
transport [mmol gDW^-1 h^-1]: 4.365741867082851e-05
out [mmol gDW^-1 h^-1]: 4.365741867082851e-05


In [41]:
with model:
    model.objective = target
    print("every reaction going from Typtophan")
    
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    print("TRPTRS [mmol gDW^-1 h^-1]:",model.reactions.TRPTRS.flux)
    print("TRPS2 [mmol gDW^-1 h^-1]:",model.reactions.TRPS2.flux)
    print("CrTdc [mmol gDW^-1 h^-1]:",model.reactions.CrTdc.flux)
    print("TRPS1 [mmol gDW^-1 h^-1]:",model.reactions.TRPS1.flux)
    print("TRPt2rpp [mmol gDW^-1 h^-1]:",model.reactions.TRPt2rpp.flux)
    print("TRPabcpp [mmol gDW^-1 h^-1]:",model.reactions.TRPabcpp.flux)
    print("PROTEIN [mmol gDW^-1 h^-1]:",model.reactions.PROTEIN.flux)
    

every reaction going from Typtophan
Max. psi production [mmol gDW^-1 h^-1]: 4.365741867082851e-05
TRPTRS [mmol gDW^-1 h^-1]: 0.0
TRPS2 [mmol gDW^-1 h^-1]: 0.0
CrTdc [mmol gDW^-1 h^-1]: 0.096979888828
TRPS1 [mmol gDW^-1 h^-1]: 0.1
TRPt2rpp [mmol gDW^-1 h^-1]: 0.0
TRPabcpp [mmol gDW^-1 h^-1]: 0.0
PROTEIN [mmol gDW^-1 h^-1]: 0.054028


In [42]:
print(f'{len(model.reactions)} reactions initially')
print(f'{len(model.metabolites)} metabolites initially')
print(f'{len(model.genes)} genes initially')

1222 reactions initially
956 metabolites initially
773 genes initially
